# Blue Brain BioExplorer

![](../bioexplorer_banner.png)

## Visualization of a membrane defined by a mesh

In [ ]:
from bioexplorer import BioExplorer, Cell, Protein, Membrane, RandomizationParams, Vector2, Vector3, Quaternion
import glob

be = BioExplorer('localhost:5000')
core = be.core_api()
status = be.reset_scene()

### Resources

In [ ]:
resource_folder = '../../tests/test_files/'
pdb_folder = resource_folder + 'pdb/'
obj_folder = resource_folder + 'obj/'
membrane_folder = pdb_folder + 'membrane/'
lipids_folder = membrane_folder + 'lipids/'
transporters_folder = pdb_folder + 'transporters/'

representation = be.REPRESENTATION_ATOMS_AND_STICKS

### Configuration

In [ ]:
mesh_source= obj_folder + 'suzanne.obj'
rotation = Quaternion(1, 0, 0, 0)
position = Vector3(100.0, 0.0, 0.0)
scale = Vector3(2.5,2.5,2.5)

## Camera position

In [ ]:
status = be.core_api().set_camera(
    orientation=[-0.114, 0.341, 0.020, 0.932],
    position=[368.152, 107.242, 367.312],
    target=[50.364, -6.824, -5.159]
)

### Membrane

In [ ]:
name = 'Suzanne'

# ACE2 receptor definition
ace2_receptor = Protein(
    name=be.NAME_TRANS_MEMBRANE + '_ACE2',
    source=pdb_folder + '6m18.pdb', 
    transmembrane_params=Vector2(1.0, 2.0),
    rotation=Quaternion(0.0, 1.0, 0.0, 0.0),
    random_params=RandomizationParams(1), occurences=20)

# GLUT3 definition
transporter = Protein(
    name=be.NAME_TRANS_MEMBRANE + '_GLUT3',
    source=transporters_folder + '4zwc.pdb',
    transmembrane_params=Vector2(1.0, 2.0),
    rotation=Quaternion(0.707, 0.707, 0.0, 0.0),
    random_params=RandomizationParams(2), chain_ids=[1], occurences=30)

# Membrane definition
pdb_lipids = glob.glob(lipids_folder + '*.pdb')[:8]

membrane = Membrane(
    lipid_sources=pdb_lipids,
    load_non_polymer_chemicals=True, load_bonds=True,
    random_params=RandomizationParams(0, 1, 0.025, 2, 0.5)
)

# Cell definition
cell = Cell(
    name=name,
    shape=be.ASSEMBLY_SHAPE_MESH,
    shape_params=scale,
    shape_mesh_source=mesh_source, 
    membrane=membrane,
    proteins=[ace2_receptor, transporter]
)

# Add cell to scene
status = be.add_cell(
    cell=cell, representation=representation)

In [ ]:
be.apply_default_color_scheme(shading_mode=be.SHADING_MODE_BASIC)

## Rendering settings

In [ ]:
status = be.set_rendering_quality(be.RENDERING_QUALITY_HIGH)

## Environment with protein location constraints

In [ ]:
from bioexplorer import Protein, Volume, RandomizationParams

In [ ]:
scene_size = Vector3(300, 300, 300)
nb_glucoses = 100
nb_lactoferrins = 20

glucose_path = pdb_folder + 'glucose.pdb'
lactoferrin_path=pdb_folder + 'immune/1b0l.pdb'
defensin_path = pdb_folder + 'immune/1ijv.pdb'

In [ ]:
constraints=list()
constraints.append([be.POSITION_CONSTRAINT_OUTSIDE, 'Suzanne'])

In [ ]:
glucose = Protein(
    name=be.NAME_GLUCOSE, 
    source=glucose_path, load_non_polymer_chemicals=True, 
    random_params=RandomizationParams(1, 0, 0.0, 2, 1.0),
    occurences=nb_glucoses)
volume = Volume(
    name=be.NAME_GLUCOSE, 
    shape=be.ASSEMBLY_SHAPE_CUBE, shape_params=scene_size,
    protein=glucose)
status = be.add_volume(
    volume=volume,
    representation=be.REPRESENTATION_ATOMS_AND_STICKS,
    atom_radius_multiplier=2.0, constraints=constraints)

In [ ]:
lactoferrin = Protein(
    name=be.NAME_LACTOFERRIN, 
    source=lactoferrin_path, load_non_polymer_chemicals=True,
    random_params=RandomizationParams(3, 0, 0.0, 4, 1.0),
    occurences=nb_lactoferrins)
lactoferrins_volume = Volume(
    name=be.NAME_LACTOFERRIN, 
    shape=be.ASSEMBLY_SHAPE_CUBE, shape_params=scene_size,
    protein=lactoferrin,
)
status = be.add_volume(
    volume=lactoferrins_volume,
    representation=be.REPRESENTATION_ATOMS_AND_STICKS,
    atom_radius_multiplier=2.0, constraints=constraints)

In [ ]:
from bioexplorer import MovieMaker
mm = MovieMaker(be)

In [ ]:
mm.create_snapshot(
    renderer='bio_explorer',
    path='/tmp', base_name='suzanne',
    size=[512, 512], samples_per_pixel=64)